In this piece of code I simultaneously optimize over the unitary basis rotation and the mps tensors.

This is often done in "DMRG-SCF", but people conventionally optimize the matrix product state quite far before taking a small step in the orbital rotations. In this notebook I try to combine grassmann gradient descent with "DMRG-SCF", where I optimize the orbital basis after every linesearch. This may be more efficient...

In [1]:
using Revise,LinearAlgebra, MPSKit, TensorKit, MPSKitExperimental
using GaussianBasis,DelimitedFiles

LinearAlgebra.BLAS.set_num_threads(1);
#TensorKit.TO.disable_cache();
TensorKit.Strided.disable_threads();

In [2]:
using Base.Threads; nthreads()

4

In [3]:
bset = BasisSet("6-31g", """
N        0.00      0.00     0.00      
N        2.118      0.00     0.00""")

6-31g Basis Set
Type: Spherical   Backend: Libcint

Number of shells: 10
Number of basis:  18

N: 1s 2s 1p 3s 2p 
N: 1s 2s 1p 3s 2p

I will keep the first 3 orbitals frozen filled, treat the next 10 orbitals using MPS, and keep the rest frozen free.

In [4]:
N_ORBS = bset.nbas;
N_frozen_filled = 4;
N_active = 12;
N_frozen_empty = N_ORBS-N_frozen_filled-N_active;
NELEC = 14;
MS2 = 0;

The state is now smaller:

In [8]:
psp = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((0,0,0)=>1, (1,1//2,1)=>1, (2,0,0)=>1);

left = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((-(NELEC-N_frozen_filled*2),MS2//2,mod(-(NELEC-N_frozen_filled*2),2))=>1);
right = oneunit(left);

virtual = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((i,s,b)=>1 for i in -(NELEC-N_frozen_filled*2):0, s in 0:1//2:(MS2//2+1), b in (0,1));

st = FiniteMPS(rand,Float64,N_active,psp,virtual;left,right);

The code works through a weird new type. We cannot work directly with an MPO hamiltonian, because I want to consider general unitary rotations between frozen orbitals and the active space. The MPO will only describe the active space, and therefore does not contain enough information.

In [6]:
nh = CASSCF_Ham(bset,N_frozen_filled+1:N_frozen_filled+N_active);

Let's try it out!

In [11]:
# first some dmrg to ramp up the bond dimension
mpo_opp = MPSKitExperimental.mpo_representation(nh)[1]; # will be made easier in future
(st,env) = find_groundstate(st,mpo_opp,DMRG2(maxiter=1,trscheme=truncdim(100)),disk_environments(st,mpo_opp));

# grassmannscf simultaneously converges the state and the oribtals
@time (st,nh) = find_groundstate(st,nh,GrassmannSCF(maxiter=200));

┌ Info: DMRG2 iteration:
│   iter = 1
│   ϵ = 0.0024758981739543273
│   λ = -90.42386652829893
│   Δt = 6.943917881
└ @ MPSKit /home/maarten/projects/MPSKit.jl/src/algorithms/groundstate/dmrg.jl:119
┌ Warning: DMRG2 maximum iterations
│   iter = 1
│   ϵ = 0.0024758981739543273
│   λ = -90.42386652829893
└ @ MPSKit /home/maarten/projects/MPSKit.jl/src/algorithms/groundstate/dmrg.jl:123
┌ Info: DMRG2 summary:
│   ϵ = 0.0024758981739543273
│   λ = -90.42386652829893
│   Δt = 6.948388389
└ @ MPSKit /home/maarten/projects/MPSKit.jl/src/algorithms/groundstate/dmrg.jl:127


┌ Info: CG: initializing with f = -90.423866528299, ‖∇f‖ = 1.4441e-02
└ @ OptimKit /home/maarten/.julia/packages/OptimKit/xpmbV/src/cg.jl:39


┌ Info: CG: initializing with f = -90.423873777291, ‖∇f‖ = 1.9324e+01
└ @ OptimKit /home/maarten/.julia/packages/OptimKit/xpmbV/src/cg.jl:39


┌ Info: CG: iter    1: f = -97.772050938994, ‖∇f‖ = 2.4290e+00, α = 7.98e-01, β = 0.00e+00, nfg = 2
└ @ OptimKit /home/maarten/.julia/packages/OptimKit/xpmbV/src/cg.jl:83


┌ Info: CG: iter    2: f = -97.904852553886, ‖∇f‖ = 1.5571e+00, α = 4.42e-02, β = 3.50e-01, nfg = 4
└ @ OptimKit /home/maarten/.julia/packages/OptimKit/xpmbV/src/cg.jl:83


...

┌ Info: CG: initializing with f = -108.507460403533, ‖∇f‖ = 3.9405e-02
└ @ OptimKit /home/maarten/.julia/packages/OptimKit/xpmbV/src/cg.jl:39


┌ Info: CG: converged after 1 iterations: f = -108.507493026413, ‖∇f‖ = 1.2990e-02
└ @ OptimKit /home/maarten/.julia/packages/OptimKit/xpmbV/src/cg.jl:100


┌ Info: CG: iter   28: f = -108.507493026413, ‖∇f‖ = 6.3557e-02, α = 5.00e-02, β = 1.48e-01, nfg = 3
└ @ OptimKit /home/maarten/.julia/packages/OptimKit/xpmbV/src/cg.jl:83


┌ Info: CG: initializing with f = -108.507611250754, ‖∇f‖ = 4.4830e-02
└ @ OptimKit /home/maarten/.julia/packages/OptimKit/xpmbV/src/cg.jl:39


CompositeException: TaskFailedException

    nested task error: InterruptException:
    Stacktrace:
     [1] to_indices
       @ ./indices.jl:356 [inlined]
     [2] to_indices
       @ ./indices.jl:344 [inlined]
     [3] getindex(::Array{Float64, 4}, ::Int64, ::Function, ::Function, ::Vararg{Any})
       @ Base ./abstractarray.jl:1296
     [4] (::MPSKitExperimental.var"#410#421"{UnitRange{Int64}, UnitRange{Int64}, Vector{Matrix{Float64}}, Vector{Tuple{Int64, Int64}}})()
       @ MPSKitExperimental ~/projects/MPSKitExperimental.jl/src/quantumchemistry/orbopt.jl:281
     [5] lock(f::MPSKitExperimental.var"#410#421"{UnitRange{Int64}, UnitRange{Int64}, Vector{Matrix{Float64}}, Vector{Tuple{Int64, Int64}}}, l::SpinLock)
       @ Base ./lock.jl:229
     [6] macro expansion
       @ ~/projects/MPSKitExperimental.jl/src/quantumchemistry/orbopt.jl:278 [inlined]
     [7] (::MPSKitExperimental.var"#1952#threadsfor_fun#416"{MPSKitExperimental.var"#1952#threadsfor_fun#406#417"{CASSCF_Ham{BasisSet{G

In practice, one should probably start from a well informed initial guess, instead of using atomic orbitals. Nevertheless, as a proof of principle, it works.